## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..","Weather_Database","WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Upernavik,GL,2021-12-03 20:26:20,72.7868,-56.1549,13.68,81,40,7.63,scattered clouds
1,1,Miyoshi,JP,2021-12-03 20:26:21,35.0833,137.0667,49.62,68,9,2.57,clear sky
2,2,Renqiu,CN,2021-12-03 20:26:22,38.6989,116.0936,38.03,48,0,0.83,clear sky
3,3,Cabedelo,BR,2021-12-03 20:26:22,-6.9811,-34.8339,83.07,76,68,11.32,broken clouds
4,4,Cape Town,ZA,2021-12-03 20:21:31,-33.9258,18.4232,63.12,62,20,5.99,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("How cold of a trip do you want? "))
max_temp = float(input("How hot of a trip do you want? "))

How cold of a trip do you want? 70
How hot of a trip do you want? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
3,3,Cabedelo,BR,2021-12-03 20:26:22,-6.9811,-34.8339,83.07,76,68,11.32,broken clouds
7,7,Saint George,US,2021-12-03 20:26:25,37.1041,-113.5841,72.07,26,1,1.01,clear sky
10,10,Atuona,PF,2021-12-03 20:26:27,-9.8000,-139.0333,78.51,73,6,17.90,clear sky
11,11,Cidreira,BR,2021-12-03 20:26:28,-30.1811,-50.2056,70.20,72,23,19.19,few clouds
14,14,Victoria Point,AU,2021-12-03 20:26:29,-27.5833,153.3000,74.08,93,5,1.01,clear sky
18,18,Clarence Town,AU,2021-12-03 20:26:31,-32.5833,151.7833,70.34,95,99,2.64,overcast clouds
27,27,Indramayu,ID,2021-12-03 20:26:34,-6.3264,108.3200,81.46,77,100,7.72,very heavy rain
28,28,Saint-Philippe,RE,2021-12-03 20:26:34,-21.3585,55.7679,78.35,86,19,10.16,few clouds
30,30,Araguacu,BR,2021-12-03 20:26:35,-12.9306,-49.8264,84.58,69,57,3.44,light rain
34,34,Manokwari,ID,2021-12-03 20:26:37,-0.8667,134.0833,77.95,84,100,8.10,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.isnull().sum()

266


City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_preferred_cities_df=preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_preferred_cities_df))
clean_preferred_cities_df.head()

266


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
3,3,Cabedelo,BR,2021-12-03 20:26:22,-6.9811,-34.8339,83.07,76,68,11.32,broken clouds
7,7,Saint George,US,2021-12-03 20:26:25,37.1041,-113.5841,72.07,26,1,1.01,clear sky
10,10,Atuona,PF,2021-12-03 20:26:27,-9.8000,-139.0333,78.51,73,6,17.90,clear sky
11,11,Cidreira,BR,2021-12-03 20:26:28,-30.1811,-50.2056,70.20,72,23,19.19,few clouds
14,14,Victoria Point,AU,2021-12-03 20:26:29,-27.5833,153.3000,74.08,93,5,1.01,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
3,Cabedelo,BR,83.07,broken clouds,-6.9811,-34.8339,
7,Saint George,US,72.07,clear sky,37.1041,-113.5841,
10,Atuona,PF,78.51,clear sky,-9.8000,-139.0333,
11,Cidreira,BR,70.20,few clouds,-30.1811,-50.2056,
14,Victoria Point,AU,74.08,clear sky,-27.5833,153.3000,
18,Clarence Town,AU,70.34,overcast clouds,-32.5833,151.7833,
27,Indramayu,ID,81.46,very heavy rain,-6.3264,108.3200,
28,Saint-Philippe,RE,78.35,few clouds,-21.3585,55.7679,
30,Araguacu,BR,84.58,light rain,-12.9306,-49.8264,
34,Manokwari,ID,77.95,overcast clouds,-0.8667,134.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

  
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index,"Hotel Name"]=np.nan        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
print(len(hotel_df))
hotel_df.dtypes
hotel_df.isna().sum()

clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df

266


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
3,Cabedelo,BR,83.07,broken clouds,-6.9811,-34.8339,Hotel Pigalle
7,Saint George,US,72.07,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
10,Atuona,PF,78.51,clear sky,-9.8000,-139.0333,Villa Enata
11,Cidreira,BR,70.20,few clouds,-30.1811,-50.2056,Hotel Castelo
14,Victoria Point,AU,74.08,clear sky,-27.5833,153.3000,Bay Retreat Motel
...,...,...,...,...,...,...,...
665,Barabai,ID,74.97,overcast clouds,-2.5833,115.3833,HOTEL BOURAQ INDAH PERMAI
669,Betioky,MG,77.76,clear sky,-23.7167,44.3833,Bungalows Acacias Hotel
671,Kendari,ID,75.56,overcast clouds,-3.9450,122.4989,Hotel Wixel
676,Miranda,CO,77.13,overcast clouds,3.2499,-76.2281,unidiversidad organica la utopia


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig



Figure(layout=FigureLayout(height='420px'))